<a href="https://colab.research.google.com/github/pedroteche-ih/99_PT_JAN2023/blob/main/aulas/20230530%20Deep%20Learning%20com%20Keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.model_selection import train_test_split

# Deep Learning

## Modelos de Classificação

In [ ]:
url = 'https://raw.githubusercontent.com/pedroteche-ih/64_PT_NOV202111/main/aulas/data/tb_hotel_completa.csv'
tb_hotel = pd.read_csv(url)
tb_hotel['is_company'] = np.where(tb_hotel['company'].isna(), 0, 1)
tb_hotel['is_agent'] = np.where(tb_hotel['agent'].isna(), 0, 1)

In [ ]:
tb_hotel.info()

## Construindo Pipelines

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [ ]:
tb_hotel.select_dtypes(include = "number").columns

In [ ]:
cat_vars = [
    'hotel', 'meal', 'country', 
    'market_segment', 'distribution_channel',
    'reserved_room_type', 'assigned_room_type', 
    'deposit_type', 'customer_type', 'is_company', 'is_agent'
]
num_vars = [
    'lead_time', 'stays_in_weekend_nights',
    'stays_in_week_nights', 'adults', 'children', 'babies',
    'is_repeated_guest', 'previous_cancellations',
    'previous_bookings_not_canceled','days_in_waiting_list', 
    'adr', 'required_car_parking_spaces',
    'total_of_special_requests'
]

#### Braço Numérico

In [ ]:
num_imputer = KNNImputer(n_neighbors = 5, weights = 'distance')
num_scaler = StandardScaler()
num_pipeline = Pipeline([('IMPUTER', num_imputer), ('SCALER', num_scaler)])


#### Braço Categórico

In [ ]:
cat_imputer = SimpleImputer(strategy = 'constant', fill_value = 'Unknown')
ohe = OneHotEncoder(
    drop = 'first', 
    handle_unknown = 'ignore',
    min_frequency = 10
)
cat_pipeline = Pipeline([('IMPUTER', cat_imputer), ('OHE', ohe)])

#### Pipeline Final

In [ ]:
composed = ColumnTransformer([('CAT', cat_pipeline, cat_vars),
                              ('NUM', num_pipeline, num_vars)])
dataprep_pipeline = Pipeline([('DATAPREP', composed)])

In [ ]:
X = tb_hotel[cat_vars + num_vars]
y = np.array(tb_hotel['is_canceled'])
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
dataprep_pipeline.fit(X_train)
X_train_trans = dataprep_pipeline.transform(X_train)
X_test_trans = dataprep_pipeline.transform(X_test)

In [ ]:
num_features = X_train_trans.shape[1]
print(num_features)

## Criando nossa Rede Neural

In [ ]:
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout

In [ ]:
%load_ext tensorboard

#### Definindo a topologia

Na definição da topologia precisamos prestar atenção à alguns parametros:

1. `input_dim` na primeira camada deve ser o número de features em nosso modelo;
1. `activation` nas camadas escondidas é hiperparâmetro a ser testado;
1. `activation` na última camada é uma função do tipo de modelo que queremos construir:
  * "sigmoid" para classificação binária;
  * "softmax" para multi-classificação;
  * não precisa ser especificada para problemas de regressão;
1. o **tamanho da última camada** deve corresponder ao tipo de previsão que queremos fazer:
  * **1** para problemas de classificação binária;
  * **n** para problemas de classificação com *n* categorias;
  * **1** para problemas de regressão.

Além das camadas densas (como as camadas de um MLP) vamos adicionar uma camada de `Dropout`: 

1. Camadas de *dropout* tem uma probabilidade de não passar as informações de alguns neurônios de uma camada para a próxima - essa técnica ajuda redes profundas a evitar overfitting.
1. O único hiperparâmetro de uma cada `Dropout` é *p*, a % de neurônios que são bloqueados em cada batch.

In [ ]:
model = Sequential()
model.add(Dense(30, input_dim=num_features, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(20, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(10, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(1, activation="sigmoid"))

Agora podemos compilar nosso modelo. O parâmetro fundamental dessa função é a *loss function*, que é consequência do problema que queremos resolver:

1. `binary_crossentropy` para problemas de classificação binária;
1. `categorical_crossentropy` para problemas de multi-classificação;
1. `mean_squared_error` para problemas de regressão.

https://keras.io/api/losses/

De forma semelhante, a métrica utilizada precisa refletir o tipo de problema que estamos resolvendo:

https://keras.io/api/metrics/

In [ ]:
model.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=['accuracy'])

#### Estimando pesos

O principal hiperparâmetro em um modelo de Deep Learning é o número de *épocas* (`epochs`) - que representa por quantas rodadas de otimização nossa rede passará. Um número maior de épocas melhora nosso modelo mas pode causar overfitting!

In [ ]:
h = model.fit(X_train_trans.toarray(), y_train, epochs = 25, batch_size = 64, validation_split = 0.1)

In [ ]:
y_pred_prob = model.predict(X_test_trans)
y_pred_prob

In [ ]:
y_pred = [1 if x > 0.5 else 0 for x in y_pred_prob]
y_pred[0:10]

In [ ]:
from sklearn.metrics import f1_score
print(f"F1-Scoreo no Conjunto de Teste: {np.round(f1_score(y_test, y_pred), 3)}")

### Visualizando Overfitting

Para visualizar overfitting, vamos criar uma rede profunda, com mais camadas:

In [ ]:
model_2 = Sequential()
model_2.add(Dense(80, input_dim=num_features, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(80, activation="relu"))
model_2.add(Dropout(0.1))
model_2.add(Dense(1, activation="sigmoid"))
model_2.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

In [ ]:
history = model_2.fit(
    X_train_trans.toarray(), y_train,
    validation_split = 0.1, 
    epochs=25, batch_size=64)

In [ ]:
history.history.keys()

In [ ]:
tb_eval = pd.DataFrame({'precision' : history.history['precision'], 
                        'recall' : history.history['recall'], 
                        'val_precision' : history.history['val_precision'], 
                        'val_recall' : history.history['val_recall'], 
                        'epoch' : range(25)})

In [ ]:
tb_eval['f1_score'] = 2 * (tb_eval['precision'] * tb_eval['recall'])/(tb_eval['precision'] + tb_eval['recall'])
tb_eval['val_f1_score'] = 2 * (tb_eval['val_precision'] * tb_eval['val_recall'])/(tb_eval['val_precision'] + tb_eval['val_recall'])

In [ ]:
sns.lineplot(data = tb_eval, x = 'epoch', y = 'f1_score', label = "Train")
sns.lineplot(data = tb_eval, x = 'epoch', y = 'val_f1_score', label = "Test")

In [ ]:
y_pred_prob = model_2.predict(X_test_trans)
y_pred = [1 if x > 0.5 else 0 for x in y_pred_prob]
print(f"F1-Score no Conjunto de Teste: {np.round(f1_score(y_test, y_pred), 3)}")

# Utilizando `TensorBoard`

In [ ]:
model_3 = Sequential()

model_3.add(Dense(80, input_dim=num_features, activation="relu"))
model_3.add(Dropout(0.1))
model_3.add(Dense(80, activation="relu"))
model_3.add(Dropout(0.1))
model_3.add(Dense(80, activation="relu"))
model_3.add(Dropout(0.1))
model_3.add(Dense(1, activation="sigmoid"))

model_3.compile(
    loss='binary_crossentropy', 
    optimizer='adam', 
    metrics=[keras.metrics.Precision(), keras.metrics.Recall()])

tensorboard_callback = tf.keras.callbacks.TensorBoard('./log', histogram_freq=1)

In [ ]:
history = model_3.fit(
    X_train_trans.toarray(), y_train,
    validation_split = 0.1, 
    epochs=25, 
    batch_size=64,
    callbacks=[tensorboard_callback])

In [ ]:
y_pred_prob = model_3.predict(X_test_trans)
y_pred = [1 if x > 0.5 else 0 for x in y_pred_prob]
print(f"F1-Score no Conjunto de Teste: {np.round(f1_score(y_test, y_pred), 3)}")

In [ ]:
%tensorboard --logdir log